<a href="https://colab.research.google.com/github/maverick98/Coursera/blob/master/mfml2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from functools import reduce

In [2]:
def create_random_invertible_matrix(n=5):
    A = np.random.randint(low=0,high=10,size=(n, n))
    mx = np.sum(np.abs(A), axis=1)
    np.fill_diagonal(A, mx)
    A=A.astype(float)
    return A
def transpose(A):
    m=A.shape[0]
    n=A.shape[1]
    A_T=np.zeros((n,m))
    for i in range(n):
        A_T[i]=A[:,i]
    return A_T
def multiply(A,B):
    if A.shape[1] != B.shape[0]:
       print("Illegal multiplication")
       return
    m=A.shape[0]
    n=A.shape[1]
    p=B.shape[1]
    C=np.zeros((m,p))
    for i in range(m):
        for k in range(p):
            for j in range(n):
                C[i][k]+=A[i][j]*B[j][k]

    return C

def create_random_symmetric_positive_definite_matrix(n):
    A=create_random_invertible_matrix(n)
    A_T=transpose(A)
    return multiply(A,A_T)

def create_identity_matrix(n):
     I=np.zeros((n,n))
     for i in range(n):
         I[i][i]=1
     return I



#Elementary row operations
def const_multiple(A,i,c):
    A[i]*=c
def exchange_rows(A,i,j):
    A[[i,j]] = A[[j,i]]
def add_const_row_multiple(A,i,j,c):
    A[i]+=A[j]*c

def find_pivot_row(A, curr_pivot_row, col):
    rows = A.shape[0]
    for row in range(curr_pivot_row, rows):
        if A[row, col] != 0.0:
            return row
    return None




def convert_pivot_row_column_values_to_zero_below(A,pivot_row,col,elementary_matrices):
    n=A.shape[0]
    rows = A.shape[0]

    for row in range(pivot_row+1, rows):
        const_multiplier=A[row][col]/A[pivot_row][col]
        elementary_matrix=create_identity_matrix(n)
        elementary_matrix[row][col]=const_multiplier
        elementary_matrices.append(elementary_matrix)
        #print('subtracting {}*A[{}] from A[{}]'.format(const_multiplier, pivot_row,row))
        A[row] -=const_multiplier*A[pivot_row]
    return elementary_matrices

def calculate_row_echelon_form(A):
    n=A.shape[0]

    pivot_row_cols=[]
    pivot_cols=[]
    non_pivot_cols=[]
    rows=A.shape[0]
    cols=A.shape[1]
    elementary_matrices=[]
    permutation_matrices=[]


    curr_pivot_row=0
    for col in range(cols):
        nonzero_row = find_pivot_row(A, curr_pivot_row, col)
        if nonzero_row is not None:
            if curr_pivot_row != nonzero_row:
                exchange_rows(A,curr_pivot_row,nonzero_row)
                perm_matrix=create_identity_matrix(n)
                exchange_rows(perm_matrix,curr_pivot_row,nonzero_row)
                permutation_matrices.append(perm_matrix)
                perm_matrix_inverse=transpose(perm_matrix)
                elementary_matrices.append(perm_matrix_inverse)
            pivot_row_cols.append((curr_pivot_row,col))
            convert_pivot_row_column_values_to_zero_below(A,curr_pivot_row,col,elementary_matrices)
            curr_pivot_row+=1
        else:
            print('Stopping...')
            if len(elementary_matrices) == 0:
               elementary_matrices.append(create_identity_matrix(n))
            return elementary_matrices,permutation_matrices


    return elementary_matrices,permutation_matrices

def multiply_all_matrices(matrices):
    n=matrices[0].shape[0]
    I=create_identity_matrix(n)
    #product = reduce(lambda x, y: multiply(x,y), matrices,I)
    product=I
    for matrix in matrices:
        product=multiply(product,matrix)
    return product

In [29]:

import copy
def calculate_LU_Decomposition(A_original):
    print('Input matrix is ')
    print(A_original)
    print('---------')
    A=copy.deepcopy(A_original)
    elementary_matrices,permutation_matrices=calculate_row_echelon_form(A)
    print('U is')
    print(A)
    print('-----')
    L=multiply_all_matrices(elementary_matrices)
    L_modified=L
    print('------')
    if len(permutation_matrices) >0:

       for permutation_matrix in permutation_matrices:
           L_modified=multiply(permutation_matrix,L_modified)

    L=L_modified
    print("L is")
    print(L)
    print('------')

    if len(permutation_matrices) >0:
        print('perm matrices are {}'.format(len(permutation_matrices)))
        P=multiply_all_matrices(permutation_matrices)
        print(P)
        print('**********')




In [ ]:
A = np.array([[ 1 ,1,1 ], [1,1,3], [2,5,8]])
A=A.astype(float)
calculate_LU_Decomposition(A)


In [ ]:
A = np.array([[ 10 ,45,0 ], [10,34,0], [10,118,0]])
A=A.astype(float)
calculate_LU_Decomposition(A)


In [ ]:
A = np.array([[ 10 ,0,45 ], [10,0,34], [10,0,118]])
A=A.astype(float)
calculate_LU_Decomposition(A)


In [ ]:
A = np.array([[ 0,10,45 ], [0,10,34], [0,10,118]])
A=A.astype(float)
calculate_LU_Decomposition(A)

In [30]:
A=create_random_symmetric_positive_definite_matrix(n=3)
A

array([[185., 166.,  67.],
       [166., 377., 191.],
       [ 67., 191., 131.]])

In [31]:
calculate_LU_Decomposition(A)

Input matrix is 
[[185. 166.  67.]
 [166. 377. 191.]
 [ 67. 191. 131.]]
---------
U is
[[185.         166.          67.        ]
 [  0.         228.04864865 130.88108108]
 [  0.           0.          31.62020906]]
-----
------
L is
[[1.         0.         0.        ]
 [0.8972973  1.         0.        ]
 [0.36216216 0.57391737 1.        ]]
------


In [32]:
def cholesky_decomposition(A):
    n=A.shape[0]
    L=np.zeros((n,n)).astype(float)
    for i in range(n):
        for j in range(i+1):
            if i==j:
               l_sum=0
               for k in range(j):
                    l_sum+=L[j][k]*L[j][k]
               L[i][j]=np.sqrt(A[i][j]-l_sum)
            else:
                l_sum=0
                for k in range(j):
                    l_sum+=L[i][k]*L[j][k]
                L[i][j]= (A[i][j]-l_sum)/L[j][j]
    return L,transpose(L)


In [33]:
#A = np.array([[ 4,12,-16 ], [12,37,-43], [-16,-43,98]])
#A=A.astype(float)
calculate_LU_Decomposition(A)
L,L_T=cholesky_decomposition(A)
L,L_T

Input matrix is 
[[185. 166.  67.]
 [166. 377. 191.]
 [ 67. 191. 131.]]
---------
U is
[[185.         166.          67.        ]
 [  0.         228.04864865 130.88108108]
 [  0.           0.          31.62020906]]
-----
------
L is
[[1.         0.         0.        ]
 [0.8972973  1.         0.        ]
 [0.36216216 0.57391737 1.        ]]
------


(array([[13.60147051,  0.        ,  0.        ],
        [12.20456273, 15.1012797 ,  0.        ],
        [ 4.92593797,  8.66688676,  5.62318496]]),
 array([[13.60147051, 12.20456273,  4.92593797],
        [ 0.        , 15.1012797 ,  8.66688676],
        [ 0.        ,  0.        ,  5.62318496]]))

In [36]:
product=multiply_all_matrices([L,L_T])
print('L*L_T is')
print(product)
print('Original matrix was ')
print(A)

L*L_T is
[[185. 166.  67.]
 [166. 377. 191.]
 [ 67. 191. 131.]]
Original matrix was 
[[185. 166.  67.]
 [166. 377. 191.]
 [ 67. 191. 131.]]


In [12]:
class Vector:
    def __init__(self,values):
        self.values=values
        self.dim=len(self.values)

    def norm(self):
        return np.sqrt(np.sum(self.values**2))

    def dot_product(self,that_vector):
        return np.sum(self.values*that_vector.values)

    def angle(self,that_vector):
        cos_theta=self.dot_product(that_vector)/(that_vector.norm()*self.norm())
        return 180*np.arccos(cos_theta)/np.pi

    def const_multiplier(self,c):

        copy_values=self.values*c
        my_copy=Vector(copy_values)
        return my_copy

    def project_to(self,that_vector):
        multiplier=self.dot_product(that_vector)/(that_vector.norm()**2)
        return that_vector.const_multiplier(multiplier)

    def subtract(self,that_vector):
        return Vector(np.subtract(self.values,that_vector.values))

    def get_unit_vector(self):
        return self.const_multiplier(1.0/self.norm())
    def __str__(self):
        print(','.join(str(self.values)))

a=Vector(np.array([3,4]))
b=Vector(np.array([-4,3]))
print(a.angle(b))
c=a.subtract(b)
#a.values,b.values,c.values
e=a.get_unit_vector()
e.values


90.0


array([0.6, 0.8])

In [61]:
def calculate_QR_decomposition(A):
    m=A.shape[0]
    n=A.shape[1]
    Q=np.zeros((m,n))
    orthonormal_vectors=[]
    def take_out(q_v):
        if len(orthonormal_vectors) ==0:
            orthonormal_vectors.append(q_v.get_unit_vector())
            return

        for i in range(len(orthonormal_vectors)):
            that_vector=orthonormal_vectors[i]
            q_v=q_v.subtract(q_v.project_to(that_vector))
        orthonormal_vectors.append(q_v.get_unit_vector())


    for i in range(n):
        q=A[:,i]
        q_v=Vector(q)
        take_out(q_v)
        #print(orthonormal_vectors[i].values)
        Q[:,i]=orthonormal_vectors[i].values
    Q_T=transpose(Q)

    R=multiply(transpose(Q),A)
    return Q,R
A = np.array([[ 1 ,-6 ], [4,-9],[7,6]])
print(A)
calculate_QR_decomposition(A)



[[ 1 -6]
 [ 4 -9]
 [ 7  6]]


(array([[ 0.12309149, -0.48507125],
        [ 0.49236596, -0.72760688],
        [ 0.86164044,  0.48507125]]),
 array([[ 8.12403840e+00, -8.88178420e-16],
        [ 8.88178420e-16,  1.23693169e+01]]))

In [14]:
A=create_random_symmetric_positive_definite_matrix(n=3)


In [15]:
print(A)

[[ 96. 108. 132.]
 [108. 210. 180.]
 [132. 180. 330.]]


In [16]:
calculate_QR_decomposition(A)

(array([[ 0.49051147, -0.5495378 , -0.67631849],
        [ 0.55182541,  0.79654134, -0.24700328],
        [ 0.67445327, -0.25205179,  0.69396158]]),
 array([[ 1.95714077e+02,  2.90260163e+02,  3.86645667e+02],
        [ 8.52651283e-14,  6.25542772e+01, -1.23386381e+01],
        [-4.26325641e-14, -7.10542736e-14,  9.52726919e+01]]))

In [69]:
B=create_random_symmetric_positive_definite_matrix(n=5)
print(B)
U,_,_=np.linalg.svd(B)
a=Vector(U[:,0])
b=Vector(U[:,4])
C=U[:,[0,1,2,3]]
print("random 5 x 4 matrix having all its columns as linearly independent is as follows")
print(C)
print(C.shape)
Q,R=calculate_QR_decomposition(C)
print("Q is")
print(Q)
print("R is")
print(R)
print("Q*R gives back original matrix C")
print(multiply(Q,R))
_,Sigma,_=np.linalg.svd(U)
print("Sigma of U is ")
print(Sigma)


[[ 638.  268.  289.  320.  256.]
 [ 268.  607.  229.  151.  292.]
 [ 289.  229.  459.  170.  402.]
 [ 320.  151.  170.  512.  328.]
 [ 256.  292.  402.  328. 1204.]]
random 5 x 4 matrix having all its columns as linearly independent is as follows
[[-0.39147803  0.60401471 -0.25609444  0.2904493 ]
 [-0.34771551  0.33392172  0.7764261  -0.4041642 ]
 [-0.37124671  0.0887041   0.11173508  0.66585565]
 [-0.34121773  0.2356198  -0.56422846 -0.55393318]
 [-0.68672063 -0.67843747 -0.02719677 -0.04565941]]
(5, 4)
Q is
[[-0.39147803  0.60401471 -0.25609444  0.2904493 ]
 [-0.34771551  0.33392172  0.7764261  -0.4041642 ]
 [-0.37124671  0.0887041   0.11173508  0.66585565]
 [-0.34121773  0.2356198  -0.56422846 -0.55393318]
 [-0.68672063 -0.67843747 -0.02719677 -0.04565941]]
R is
[[ 1.00000000e+00  1.66533454e-16  2.77555756e-17  1.38777878e-17]
 [-5.55111512e-17  1.00000000e+00  9.71445147e-17 -1.42247325e-16]
 [-2.08166817e-17  6.93889390e-18  1.00000000e+00  5.59448321e-17]
 [ 1.38777878e-17  3.12

90.0